In [1]:
from kg_get import Get_kg
from get_csv import DataProcess, SequenceEncoder, GenresEncoder, IdentityEncoder
import torch
import torch.nn as nn
from torch_geometric.data import HeteroData
import pandas as pd


c:\Users\24717\.conda\envs\graph_learn\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
Kg_triples_withIds, Kg_relation_toId, Kg_entity_num= Get_kg('../data/movie/kg.txt')
Kg_data = HeteroData()
Kg_data['triples_withIds'] = Kg_triples_withIds
Kg_data['relation_toId'] = Kg_relation_toId



知识图谱统计信息:
三元组总数: 499474
关系类型数: 32
实体总数: 102569

关系分布:
    Relation ID   Count
1             1  277059
12           31   33385
0            20   21624
15           27   20967
14           14   17359
3            19   16473
9            29   13525
13            4   13304
6            26   12567
11           24   12546
4             7   11341
16           28   11136
7            22   10869
10           10    8143
2            17    6315
8            25    5832
5             2    5493
19            8     380
20            3     380
21           13     234
17           30     143
18           15     110
25            9     110
22           11     108
23           18      54
27           12       6
28           23       4
26            5       2
29           16       2
24           21       1
30           32       1
31            6       1

关系映射表:
关系: film.film.actor, ID: 1
关系: film.film.set_designer, ID: 2
关系: award.nominated_work.nomination, ID: 3
关系: film.film.star, ID: 4
关系: film.film.ca

In [3]:

movie_path = '../data/ml-latest-small/movies.csv'
rating_path = '../data/ml-latest-small/ratings.csv'
print(pd.read_csv(movie_path).head())
print(pd.read_csv(rating_path).head())

data_process = DataProcess()
_, users_mapping = data_process.load_node_csv(rating_path, index_col='userId')
items_x, items_mapping = data_process.load_node_csv(
    movie_path, index_col='movieId', encoders={
        'title': SequenceEncoder(),
        'genres': GenresEncoder()
    })

edge_index, edge_label = data_process.load_edge_csv(
    rating_path,
    src_index_col='movieId',
    src_mapping=items_mapping,
    dst_index_col='userId',
    dst_mapping=users_mapping,
    encoders={'rating': IdentityEncoder(dtype=torch.long)},
)

Interaction_data = HeteroData()
Interaction_data['users'].mapping = users_mapping
Interaction_data['items'].x = items_x
Interaction_data['items'].mapping = items_mapping
Interaction_data['edge'].edge_index = edge_index
Interaction_data['edge'].edge_label = edge_label

num_node = len(users_mapping)+len(items_mapping)
user_emb = nn.Embedding(num_embeddings=len(users_mapping)+1, embedding_dim=items_x.size(dim=1))
nn.init.normal_(user_emb.weight, std=0.1)
user_x=torch.arange(1,len(users_mapping)+1).view(1,-1)
user_x = torch.LongTensor(user_x)
user_x = user_emb(user_x)
user_x = user_x.squeeze(0)
Interaction_data['users'].x = user_x


   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


Batches: 100%|██████████| 305/305 [00:03<00:00, 82.22it/s] 


In [5]:
Kg_entity_emmbedding = nn.Embedding(num_embeddings=Kg_entity_num+1, embedding_dim=items_x.size(dim=1))
nn.init.normal_(Kg_entity_emmbedding.weight, std=0.1)
Kg_entity_x=torch.arange(1,Kg_entity_num+1).view(1,-1)
Kg_entity_x = torch.LongTensor(Kg_entity_x)
Kg_entity_x = Kg_entity_emmbedding(Kg_entity_x)
Kg_entity_x = Kg_entity_x.squeeze(0)
Kg_data['entities'].x = Kg_entity_x
